### Fonction qui implémente la résolution des sous-problèmes

Les sous-problèmes s'écrivent ainsi :

max $w_c - \sum_{i \in V} \pi_i \sum_{j : (i,j) \in A} x_{ij}$

tel que :
- $\sum_{(i,j) \in A} x_{ij} \le L$,
- $\sum_{i : (i,j) \in A} x_{ij} = 1$,
- $\sum_{j : (i,j) \in A} x_{ij} = 1$,
- $x_{ij} \in \{0,1\}$.

In [2]:
function subproblem(π,node)
    SP=Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(SP, "OutputFlag", 0)
    
    # Variable
    @variable(SP,x[i in 1:P,j in 1:P],Bin)
    
    # Contraintes
    # Le cycle doit être de longueur maximale L
    @constraint(SP,sum(x[i,j] for i in 1:P,j in 1:P if (new_inst.edge_weight[j,i]==1.0))<=L)
    # Chaque sous-problème doit représenter un cycle
    @constraint(SP,cyclej[i in 1:P],sum(x[i,j] for j in 1:P if (new_inst.edge_weight[i,j]==1.0))<=L)
    @constraint(SP,cyclei[j in 1:P],sum(x[i,j] for i in 1:P if (new_inst.edge_weight[i,j]==1.0))<=L)
    
    # Objectif
    @objective(SP,Max,sum(x[i,j] for i in 1:P,j in 1:P if (new_inst.edge_weight[j,i]==1.0))-sum(π[i]*x[i,j] for i in 1:P,j in 1:P if (new_inst.edge_weight[j,i]==1.0)))

    for (i,j) in tree[node].setzero
        @constraint(SP,x[i,j]<=0)
    end
    
    for (i,j) in tree[node].setone
        @constraint(SP,x[i,j]>=1)
    end

    optimize!(SP)

    # Si le sous-problème est résolu à l'optimalité, on retourne l'objectif et le motif optimal
    if JuMP.termination_status(SP)==MOI.OPTIMAL
        return JuMP.objective_value(SP),JuMP.value.(x)
    # Sinon, le sous-problème est infaisable et on retourne Inf, []
    else
        return Inf,[]
    end
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[2]:6